<a href="https://colab.research.google.com/github/MatthieuHanania/PlasticDetection/blob/main/Dowloading_a_model_whithout_retrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.14.0

In [ ]:
#object detection is not installed,
from object_detection.utils import visualization_utils as viz_utils

In [2]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4305, done.
remote: Counting objects: 100% (4305/4305), done.
remote: Compressing objects: 100% (3291/3291), done.
remote: Total 4305 (delta 1210), reused 2182 (delta 941), pack-reused 0 (from 0)
Receiving objects: 100% (4305/4305), 53.16 MiB | 15.36 MiB/s, done.
Resolving deltas: 100% (1210/1210), done.
Updating files: 100% (3875/3875), done.


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# Download a model
# model zoo : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
#!wget  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d5_coco17_tpu-32.tar.gz
!tar -xf efficientdet_d5_coco17_tpu-32.tar.gz
!mv efficientdet_d5_coco17_tpu-32/ models/mymodel/

In [ ]:
import io
import os
import cv2
import scipy.misc
import numpy as np
import six
import time
import pandas as pd

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

In [ ]:
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('models/mymodel/saved_model/')

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Load the COCO Label Map

category_index = {i: {'id': i, 'name': f'Class {i}'} for i in range(1, 91)}
category_index[38] = {'id': 38, 'name': 'bottle'}
category_index[44] = {'id': 44, 'name': 'plastic waste'}

In [ ]:
def export_img(image_path,image_np_with_detections, score):
    result_folder = "results/"

    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    image_name = "result_"+str(score)+"_"+os.path.basename(image_path)
    savepath = os.path.join(result_folder, image_name)
    print(savepath)

    cv2.imwrite(savepath, image_np_with_detections)


In [ ]:
import glob

test_images_folder = 'test_images'
predictions = {}

images = glob.glob(test_images_folder+"/*.jpg")

id_to_keep = [38,44]

#Parcourir chaque image et appeler le modèle
for image_to_test_path in images:
  image_np = load_image_into_numpy_array(image_to_test_path)
  input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  # On sort ce qu'il a détecté
  detection_boxes = detections['detection_boxes'][0].numpy(),
  detection_classes = detections['detection_classes'][0].numpy().astype(np.int32),
  detection_scores = detections['detection_scores'][0].numpy(),

  # On sauveagrde les classes et score dans un dataframe
  predictions[image_to_test_path] = pd.DataFrame({
    'Class ID': detection_classes,
    'Score': detection_scores
  })

  ## On souhaite garder que certaines classes, alors on filtre
  filtered_boxes = []
  fildered_classes = []
  filtered_scores = []

  detection_classes = detection_classes[0]
  detection_boxes = detection_boxes[0]
  detection_scores = detection_scores[0]

  for i in range (len(detection_classes)):
    if detection_classes[i] in id_to_keep:
      filtered_boxes.append(detection_boxes[i])
      fildered_classes.append(detection_classes[i])
      filtered_scores.append(detection_scores[i])


  # On affiche que les données filtrées
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        np.array(filtered_boxes),
        np.array(fildered_classes),
        np.array(filtered_scores),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.40,
        agnostic_mode=False)

  score = round(detections['detection_scores'][0][0].numpy(), 2)

  predictions[image_to_test_path] = pd.DataFrame({
    'Class ID': detections['detection_classes'][0].numpy().flatten(),
    'Score': detections['detection_scores'][0].numpy().flatten()
})

  export_img(image_to_test_path,image_np_with_detections,score)

In [ ]:
assert False

In [ ]:
def clean_folder(folder_path):
   for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path) : os.remove(item_path)

clean_folder("test_images")

In [ ]:
import shutil
shutil.make_archive("mymodel", 'zip', "models/mymodel")

Note

Le modèle identifie des objets, il va les catégoriser tout seul, 1,2,3,... Pour cela il se base sur le dataset COCO, 1 = voiture...

Il va ensuite regarder notre carte de catégories données pour regarder l'association données. Si on lui dit que 1 = déchets, tous les 1 seront alors indiqués comme déchets, même si lui pense que ce sont des voitures